In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
import pyspark.sql.functions as F
import json
import itertools

conf = SparkConf().setAppName('MyFirstStandaloneApp')
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

In [2]:
def scrap_time_change(t : str):
    return int(t.split(':')[0])

def flights_hours_change(t : str):
    tt = scrap_time_change(t)
    if (tt < 6):
        return 'night'
    elif (tt < 12):
        return 'morning'
    elif (tt < 18):
        return 'afternoon'
    elif (tt < 24):
        return 'evening'

In [82]:
filepath = "2019-05-15.txt,2019-05-16.txt,2019-05-17.txt,2019-05-18.txt,2019-05-19.txt"

country_from = 'Austria'
country_to = 'Italy'

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
filtered = flightsRDD.filter(lambda line: line != header)
#filtered = filtered.filter(lambda line : (line.split(';')[2] == country_from) & (line.split(';')[3] == country_to))
parts = filtered.map(lambda line: line.split(';'))


flights = parts.map(lambda p:
                   Row(Scrap_date = p[0],
                       Scrap_time = scrap_time_change(p[1]),
                       Country_from = p[2],
                       Country_to = p[3],
                       Flight_id = int(p[4]),
                       There_or_Back = p[5],
                       Flight_date = p[6],
                       Airline = p[7],
                       Change = int(p[8]),
                       Price = float(p[9]),
                       Depart_hour = flights_hours_change(p[10]),
                       Depart_from = p[11],
                       Arrival_hour = flights_hours_change(p[12]),
                       Arrive_to = p[13]
                      ))

sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(flights)
dt.registerTempTable("flights")

timeFmt1 = "yyyy-MM-dd"
timeFmt2 = "dd/MM/yy"
timeDiff = ((F.unix_timestamp('Flight_date', format=timeFmt2)
            - F.unix_timestamp('Scrap_date', format=timeFmt1)) / (24 * 3600)).cast(IntegerType())
dt = dt.withColumn("Days", timeDiff)

tt = (F.unix_timestamp('Flight_date', format=timeFmt2) / (24 * 3600)).cast(IntegerType())
dt = dt.withColumn("Flight_date_val", tt)
#dt = dt.drop(*['Flight_date', 'Scrap_date'])
df = dt

Tutaj, chce porownac jaka tabele co jest wyswietlana z tabela df. Tak, że jezeli "Scrap_date", "Scrap_time", "Flight_id", "There_or_back" w temp są takie same jak w df to wtedy do df w tym miejscu jest dodawana count w nowej kolumnie

In [80]:
temp = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id", "There_or_back"]).count().orderBy(
["Scrap_date", "Scrap_time", "Flight_id", "There_or_back"]).show()
#df = df.withColumn('Change_temp', F.lit(0))
#df = df.agg(F.sum(df.Price).alias("Price_sum"))
#df.show()

+----------+----------+---------+-------------+-----+
|Scrap_date|Scrap_time|Flight_id|There_or_back|count|
+----------+----------+---------+-------------+-----+
|2019-05-15|         2|        1|         Back|    2|
|2019-05-15|         2|        1|        There|    2|
|2019-05-15|         2|        2|         Back|    2|
|2019-05-15|         2|        2|        There|    2|
|2019-05-15|         2|        3|         Back|    2|
|2019-05-15|         2|        3|        There|    2|
|2019-05-15|         2|        4|         Back|    2|
|2019-05-15|         2|        4|        There|    2|
|2019-05-15|         2|        5|         Back|    2|
|2019-05-15|         2|        5|        There|    2|
|2019-05-15|         2|        6|         Back|    2|
|2019-05-15|         2|        6|        There|    2|
|2019-05-15|         2|        7|         Back|    2|
|2019-05-15|         2|        7|        There|    2|
|2019-05-15|         2|        8|         Back|    2|
|2019-05-15|         2|     

Poniższa komórka nie działa, zadziała, kiedy do df dodamy kolumnę df, która będzie mówiła o przesiadce w danej grupie There / Back

In [ ]:
dff = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id", "There_or_back"]).agg(
    F.first(df.Days).alias("Days"),
    F.first(df.Flight_date_val).alias("Flight_date_val"), #Z tego co widzialem to przesiadki sa tego samego dnia
    F.first(df.Airline).alias('Airline1'),
    F.when(F.first(df.Change_temp) == 1, F.last(df.Airline)).otherwise('none').alias('Airline2'),
    F.first(df.Price).alias('Price1'),
    F.when(F.first(df.Change_temp) == 1, F.last(df.Price)).otherwise('none').alias('Price2'),
    F.first(df.Depart_hour).alias('Depart_hour1'),
    F.when(F.first(df.Change_temp) == 1, F.last(df.Depart_hour)).otherwise('none').alias('Depart_hour2'),
    F.first(df.Depart_from).alias('Depart_from1'),
    F.when(F.first(df.Change_temp) == 1, F.last(df.Depart_from)).otherwise('0').alias('Depart_from2'),
    F.first(df.Arrival_hour).alias('Arrival_hour1'),
    F.when(F.first(df.Change_temp) == 1, F.last(df.Arrival_hour)).otherwise('none').alias('Arrival_hour2'),
    F.first(df.Arrive_to).alias('Arrive_to1'),
    F.when(F.first(df.Change_temp) == 1, F.last(df.Arrive_to)).otherwise('none').alias('Arrive_to2')
    )

To co jest ponizej dziala ale nie w przypadku kiedy sa nierowne liczby przesiadek

In [83]:
dff = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id", "There_or_back"]).agg(
    F.first(df.Change).alias("Change"), #To w sumie chyba nie jest potrzebne
    F.first(df.Days).alias("Days"),
    F.first(df.Flight_date_val).alias("Flight_date_val"),
    F.first(df.Airline).alias('Airline1'),
    F.when(F.first(df.Change) == 1, F.last(df.Airline)).otherwise('none').alias('Airline2'),
    F.first(df.Price).alias('Price1'),
    F.when(F.first(df.Change) == 1, F.last(df.Price)).otherwise('none').alias('Price2'),
    F.first(df.Depart_hour).alias('Depart_hour1'),
    F.when(F.first(df.Change) == 1, F.last(df.Depart_hour)).otherwise('none').alias('Depart_hour2'),
    F.first(df.Depart_from).alias('Depart_from1'),
    F.when(F.first(df.Change) == 1, F.last(df.Depart_from)).otherwise('0').alias('Depart_from2'),
    F.first(df.Arrival_hour).alias('Arrival_hour1'),
    F.when(F.first(df.Change) == 1, F.last(df.Arrival_hour)).otherwise('none').alias('Arrival_hour2'),
    F.first(df.Arrive_to).alias('Arrive_to1'),
    F.when(F.first(df.Change) == 1, F.last(df.Arrive_to)).otherwise('none').alias('Arrive_to2')
    )

Do poniższego jeszcze trzeba dodać czas pobytu

In [84]:
dff_final = dff.groupBy(["Scrap_date", "Scrap_time", "Flight_id"]).agg(
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Days)).otherwise(F.last(dff.Days)).alias('Days'),
    
(F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Flight_date_val)).otherwise(F.last(dff.Flight_date_val)) -
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Flight_date_val)).otherwise(F.last(dff.Flight_date_val))).alias('Journey_time'),
    
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Airline1)).otherwise(F.last(dff.Airline1)).alias('Airline1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Airline1)).otherwise(F.last(dff.Airline1)).alias('Airline1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Airline2)).otherwise(F.last(dff.Airline2)).alias('Airline2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Airline2)).otherwise(F.last(dff.Airline2)).alias('Airline2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Price1)).otherwise(F.last(dff.Price1)).alias('Price1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Price1)).otherwise(F.last(dff.Price1)).alias('Price1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Price2)).otherwise(F.last(dff.Price2)).alias('Price2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Price2)).otherwise(F.last(dff.Price2)).alias('Price2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_hour1)).otherwise(F.last(dff.Depart_hour1)).alias('Depart_hour1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_hour1)).otherwise(F.last(dff.Depart_hour1)).alias('Depart_hour1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_hour2)).otherwise(F.last(dff.Depart_hour2)).alias('Depart_hour2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_hour2)).otherwise(F.last(dff.Depart_hour2)).alias('Depart_hour2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_from1)).otherwise(F.last(dff.Depart_from1)).alias('Depart_from1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_from1)).otherwise(F.last(dff.Depart_from1)).alias('Depart_from1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_from2)).otherwise(F.last(dff.Depart_from2)).alias('Depart_from2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_from2)).otherwise(F.last(dff.Depart_from2)).alias('Depart_from2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrival_hour1)).otherwise(F.last(dff.Arrival_hour1)).alias('Arrival_hour1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrival_hour1)).otherwise(F.last(dff.Arrival_hour1)).alias('Arrival_hour1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrival_hour2)).otherwise(F.last(dff.Arrival_hour2)).alias('Arrival_hour2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrival_hour2)).otherwise(F.last(dff.Arrival_hour2)).alias('Arrival_hour2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrive_to1)).otherwise(F.last(dff.Arrive_to1)).alias('Arrive_to1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrive_to1)).otherwise(F.last(dff.Arrive_to1)).alias('Arrive_to1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrive_to2)).otherwise(F.last(dff.Arrive_to2)).alias('Arrive_to2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrive_to2)).otherwise(F.last(dff.Arrive_to2)).alias('Arrive_to2_Back')
)

In [ ]:
dff.show()

Poniżej jakieś moje robocze rzeczy, testy, itp.

In [60]:
dff = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id", "There_or_back"]).agg(
    F.first(df.Change).alias("Change"),
    F.first(df.Airline).alias('Airline1'),
    F.when(F.first(df.Change) == 1, F.last(df.Airline)).otherwise('none').alias('Airline2'),
    F.first(df.Price).alias('Price1'),
    F.when(F.first(df.Change) == 1, F.last(df.Price)).otherwise('none').alias('Price2')
    )

In [45]:
dff = dff.orderBy(["There_or_back"])

In [56]:
dff.filter(dff.Flight_id == 72).show()

+----------+----------+---------+-------------+------+--------+--------+------+------+
|Scrap_date|Scrap_time|Flight_id|There_or_back|Change|Airline1|Airline2|Price1|Price2|
+----------+----------+---------+-------------+------+--------+--------+------+------+
|2019-05-18|         2|       72|         Back|     1| easyJet| easyJet|  7.26| 13.67|
|2019-05-18|         2|       72|        There|     1| easyJet| easyJet| 23.42|  36.9|
|2019-05-15|         2|       72|        There|     1| easyJet| easyJet| 14.33| 26.29|
|2019-05-17|         2|       72|         Back|     1| easyJet| easyJet| 16.49| 24.87|
|2019-05-16|         2|       72|         Back|     1| easyJet| easyJet| 14.33| 25.28|
|2019-05-15|         2|       72|         Back|     1| easyJet| easyJet| 14.33| 25.28|
|2019-05-19|         2|       72|        There|     1| easyJet| easyJet| 23.42|  36.9|
|2019-05-16|         2|       72|        There|     1| easyJet| easyJet| 14.33| 26.29|
|2019-05-19|         2|       72|         B

In [42]:
dff_final = dff.groupBy(["Scrap_date", "Scrap_time", "Flight_id"]).agg(
    F.first(dff.Change).alias("Change"),
    F.first(dff.Airline1).alias("Airline1_There"),
    F.last(dff.Airline1).alias("Airline1_Back"),
    F.first(dff.Airline2).alias("Airline2_There"),
    F.last(dff.Airline2).alias("Airline2_Back"),
    F.first(dff.Price1).alias("Price1_There"),
    F.last(dff.Price1).alias("Price1_Back"),
    F.first(dff.Price2).alias("Price2_There"),
    F.last(dff.Price2).alias("Price2_Back"),
    F.first(dff.Depart_hour1).alias("Depart_hour1_There"),
    F.last(dff.Depart_hour1).alias("Depart_hour1_Back"),
    F.first(dff.Depart_hour2).alias("Depart_hour2_There"),
    F.last(dff.Depart_hour2).alias("Depart_hour2_Back"),
    F.first(dff.Depart_from1).alias("Depart_from1_There"),
    F.last(dff.Depart_from1).alias("Depart_from1_Back"),
    F.first(dff.Depart_from2).alias("Depart_from2_There"),
    F.last(dff.Depart_from2).alias("Depart_from2_Back"),
    F.first(dff.Arrival_hour1).alias("Arrival_hour1_There"),
    F.last(dff.Arrival_hour1).alias("Arrival_hour1_Back"),
    F.first(dff.Arrival_hour2).alias("Arrival_hour2_There"),
    F.last(dff.Arrival_hour2).alias("Arrival_hour2_Back"),
    F.first(dff.Arrive_to1).alias("Arrive_to1_There"),
    F.last(dff.Arrive_to1).alias("Arrive_to1_Back"),
    F.first(dff.Arrive_to2).alias("Arrive_to2_There"),
    F.last(dff.Arrive_to2).alias("Arrive_to2_Back"),
)

In [66]:
dff_final = dff.groupBy(["Scrap_date", "Scrap_time", "Flight_id"]).agg(
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Airline1)).otherwise(F.last(dff.Airline1)).alias('Airline1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Airline1)).otherwise(F.last(dff.Airline1)).alias('Airline1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Airline2)).otherwise(F.last(dff.Airline2)).alias('Airline2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Airline2)).otherwise(F.last(dff.Airline2)).alias('Airline2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Price1)).otherwise(F.last(dff.Price1)).alias('Price1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Price1)).otherwise(F.last(dff.Price1)).alias('Price1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Price2)).otherwise(F.last(dff.Price2)).alias('Price2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Price2)).otherwise(F.last(dff.Price2)).alias('Price2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_hour1)).otherwise(F.last(dff.Depart_hour1)).alias('Depart_hour1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_hour1)).otherwise(F.last(dff.Depart_hour1)).alias('Depart_hour1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_hour2)).otherwise(F.last(dff.Depart_hour2)).alias('Depart_hour2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_hour2)).otherwise(F.last(dff.Depart_hour2)).alias('Depart_hour2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_from1)).otherwise(F.last(dff.Depart_from1)).alias('Depart_from1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_from1)).otherwise(F.last(dff.Depart_from1)).alias('Depart_from1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Depart_from2)).otherwise(F.last(dff.Depart_from2)).alias('Depart_from2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Depart_from2)).otherwise(F.last(dff.Depart_from2)).alias('Depart_from2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrival_hour1)).otherwise(F.last(dff.Arrival_hour1)).alias('Arrival_hour1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrival_hour1)).otherwise(F.last(dff.Arrival_hour1)).alias('Arrival_hour1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrival_hour2)).otherwise(F.last(dff.Arrival_hour2)).alias('Arrival_hour2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrival_hour2)).otherwise(F.last(dff.Arrival_hour2)).alias('Arrival_hour2_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrive_to1)).otherwise(F.last(dff.Arrive_to1)).alias('Arrive_to1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrive_to1)).otherwise(F.last(dff.Arrive_to1)).alias('Arrive_to1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Arrive_to2)).otherwise(F.last(dff.Arrive_to2)).alias('Arrive_to2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Arrive_to2)).otherwise(F.last(dff.Arrive_to2)).alias('Arrive_to2_Back')
)

In [85]:
dff_final.show()

+----------+----------+---------+----+------------+--------------+-------------+--------------+-------------+------------+-----------+------------+-----------+------------------+-----------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|Scrap_date|Scrap_time|Flight_id|Days|Journey_time|Airline1_There|Airline1_Back|Airline2_There|Airline2_Back|Price1_There|Price1_Back|Price2_There|Price2_Back|Depart_hour1_There|Depart_hour1_Back|Depart_hour2_There|Depart_hour2_Back|  Depart_from1_There|   Depart_from1_Back|  Depart_from2_There|   Depart_from2_Back|Arrival_hour1_There|Arrival_hour1_Back|Arrival_hour2_There|Arrival_hour2_Back|    Arrive_to1_There|     Arrive_to1_Back|    Arrive_to2_There|     Arrive_to2_Back|
+----------+----------+---------+----+--

In [25]:
df.show()

+-----------+------------+--------------------+------+------------+----------+--------------------+-----------+-----------+---------+-----+----------+----------+-------------+----+---------------+
|    Airline|Arrival_hour|           Arrive_to|Change|Country_from|Country_to|         Depart_from|Depart_hour|Flight_date|Flight_id|Price|Scrap_date|Scrap_time|There_or_Back|Days|Flight_date_val|
+-----------+------------+--------------------+------+------------+----------+--------------------+-----------+-----------+---------+-----+----------+----------+-------------+----+---------------+
|Laudamotion|     evening|   Krakow KRK OE1304|     1|     Austria|   Austria|          Vienna VIE|    evening|   14/07/19|        1| 9.99|2019-05-15|         2|        There|  60|     1563055200|
|Laudamotion|     evening|          Vienna VIE|     1|     Austria|   Austria|          Krakow KRK|    evening|   14/07/19|        1| 9.05|2019-05-15|         2|        There|  60|     1563055200|
|Laudamotion|  

In [8]:
df = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id"]).agg(F.first(df.Country_from).alias("Country_from"),
                                                               F.first(df.Country_to).alias("Country_to"),
                                                               F.first(df.Change).alias("Change"),
                                                               (F.last(df.Flight_date_val) - F.first(df.Flight_date_val)).alias('Journey_time'),
                                                               F.sum(df.Price).alias("Price")
                                                              )

In [25]:
df.show()

+-----------+------------+-----------+------+------------+----------+-----------+-----------+---------+-----+----------+-------------+----+---------------+
|    Airline|Arrival_hour|  Arrive_to|Change|Country_from|Country_to|Depart_from|Depart_hour|Flight_id|Price|Scrap_time|There_or_Back|Days|Flight_date_val|
+-----------+------------+-----------+------+------------+----------+-----------+-----------+---------+-----+----------+-------------+----+---------------+
|Laudamotion|     morning|   Pisa PSA|     0|     Austria|     Italy| Vienna VIE|    morning|     1536| 9.99|         2|        There|  24|          18055|
|Laudamotion|     morning| Vienna VIE|     0|     Austria|     Italy|   Pisa PSA|    morning|     1536| 9.99|         2|         Back|  28|          18059|
|Laudamotion|     evening|   Pisa PSA|     0|     Austria|     Italy| Vienna VIE|  afternoon|     1537| 9.99|         2|        There|  33|          18064|
|Laudamotion|     evening| Vienna VIE|     0|     Austria|     I

In [50]:
d = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id", "There_or_Back"]).agg(F.count('Price').alias('Count'))
d.groupBy('Count').agg(F.max('Count').alias('Max')).show()


+-----+---+
|Count|Max|
+-----+---+
|    1|  1|
|    2|  2|
+-----+---+



In [68]:
d = df.groupBy(["Scrap_date", "Scrap_time", "Flight_id"]).agg(F.count('Price').alias('Count'))
d.groupBy('Count').count().show()

+-----+------+
|Count| count|
+-----+------+
|    3|159202|
|    2|277691|
|    4|247142|
+-----+------+



In [10]:
d.show()

+----------+----------+---------+-----+
|Scrap_date|Scrap_time|Flight_id|Count|
+----------+----------+---------+-----+
|2019-05-15|         2|       72|    4|
|2019-05-15|         2|       82|    4|
|2019-05-15|         2|      112|    4|
|2019-05-15|         2|      129|    4|
|2019-05-15|         2|      314|    3|
|2019-05-15|         2|      516|    3|
|2019-05-15|         2|      970|    3|
|2019-05-15|         2|     1036|    2|
|2019-05-15|         2|     1225|    2|
|2019-05-15|         2|     1410|    4|
|2019-05-15|         2|     2030|    3|
|2019-05-15|         2|     2213|    4|
|2019-05-15|         2|     2395|    2|
|2019-05-15|         2|     2615|    4|
|2019-05-15|         2|     2659|    4|
|2019-05-15|         2|     2759|    4|
|2019-05-15|         2|     3526|    4|
|2019-05-15|         2|     3637|    2|
|2019-05-15|         2|     3779|    3|
|2019-05-15|         2|     4728|    4|
+----------+----------+---------+-----+
only showing top 20 rows



In [ ]:
dff_final = dff.groupBy(["Scrap_date", "Scrap_time", "Flight_id"]).agg(
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Airline1)).otherwise(F.last(dff.Airline1)).alias('Airline1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Airline1)).otherwise(F.last(dff.Airline1)).alias('Airline1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Airline2)).otherwise(F.last(dff.Airline2)).alias('Airline2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Airline2)).otherwise(F.last(dff.Airline2)).alias('Airline1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Price1)).otherwise(F.last(dff.Price1)).alias('Price1_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Price1)).otherwise(F.last(dff.Price1)).alias('Price1_Back'),
F.when(F.first(dff.There_or_back) == 'There', F.first(dff.Price2)).otherwise(F.last(dff.Price2)).alias('Price2_There'),
F.when(F.first(dff.There_or_back) == 'Back', F.first(dff.Price2)).otherwise(F.last(dff.Price2)).alias('Price2_Back')
)